# Extrusion Modeling

In this tutorial illustrates another way to create simple valid boundary representation using an extrusion workflow.
The idea is to create 3D vertical features from their traces observed in 2D.

The idea of this tutorial is to illustrate another way to build boundary representations and then meshes to simulate rock physical behavior.

Also, we introduce simple random processes to generate stochastic network of fracture traces to stress the 
functionalities of [Geode-Explicit](https://docs.geode-solutions.com/references/geode-explicit/).

At the end, you will be able to create
 * random fracture traces,
 * build a fractured section,
 * extrude the section in a BRep.

In [ ]:
import opengeode as og 
import opengeode_io 
import opengeode_inspector as inspector
import geode_explicit as explicit
import geode_common as common
import geode_simplex as simplex
import geode_conversion as conversion


print("Geode module imported!")

# define various utilities functions
import os
def input(path):
  return os.path.join("data/",path) 

def create_folder_if_does_not_exists(path):
  if( not os.path.isdir(path)):
    os.makedirs(path)

def output(path):
  result_path = "results/"
  create_folder_if_does_not_exists(result_path)
  return os.path.join(result_path,path)

# define print section description
def print_section_components(section) : 
    bb =og.BoundingBox2D()
    bb.add_box(section.bounding_box())
    print("The section is centered on (" + 
          str(bb.center().value(0))+" , "+str(bb.center().value(1)) + 
          "). It is a "+str(bb.diagonal().value(0))+"m by "+
          str(bb.diagonal().value(1))+
          "m section with: ")
    print(" * "+ str(section.nb_corners()) +" corners" )
    for corner in section.corners():
        print("  - corner "+ corner.id().string()+" have " + str(corner.mesh().nb_vertices())+" vertices.") 
    print(" * "+ str(section.nb_lines()) +" lines" )
    for line in section.lines():
        print("  - line "+ line.id().string()+" have " + str(line.mesh().nb_edges())+" edges.") 
    print(" * "+ str(section.nb_surfaces()) +" surfaces" )
    for surface in section.surfaces():
        print("  - surface "+ surface.id().string()+" have " + str(surface.mesh().nb_polygons())+" triangles.") 

# define print section description
def print_brep_components(brep) : 
    bb =og.BoundingBox3D()
    bb.add_box(brep.bounding_box())
    print("The BRep is centered on (" + 
          str(bb.center().value(0))+" , "+str(bb.center().value(1)) + 
          "). It is a "+str(bb.diagonal().value(0))+"m by "+
          str(bb.diagonal().value(1))+
          "m BRep with: ")
    print(" * "+ str(brep.nb_corners()) +" corners" )
    for corner in brep.corners():
        print("  - corner "+ corner.id().string()+" have " + str(corner.mesh().nb_vertices())+" vertices.") 
    print(" * "+ str(brep.nb_lines()) +" lines" )
    for line in brep.lines():
        print("  - line "+ line.id().string()+" have " + str(line.mesh().nb_edges())+" edges.") 
    print(" * "+ str(brep.nb_surfaces()) +" surfaces" )
    for surface in brep.surfaces():
        print("  - surface "+ surface.id().string()+" have " + str(surface.mesh().nb_polygons())+" triangles.") 
    print(" * "+ str(brep.nb_blocks()) +" blocks" )
    for block in brep.blocks():
        print("  - block "+ block.id().string()+" have " + str(block.mesh().nb_polyhedra())+" polyhedra.") 

def create_aligned_box2D(extent_x,extent_y):
    bbox = og.BoundingBox2D()
    bbox.add_point(og.Point2D([0,0]))
    bbox.add_point(og.Point2D([extent_x,extent_y]))
    return bbox

def create_aligned_box3D(extent_x,extent_y,extent_z):
    bbox = og.BoundingBox3D()
    bbox.add_point(og.Point3D([0,0,0]))
    bbox.add_point(og.Point3D([extent_x,extent_y,extent_z]))
    return bbox

def create_edged_curve(points):
    curve = og.EdgedCurve2D.create()
    builder = og.EdgedCurveBuilder2D.create(curve)
    v0 = builder.create_point(points[0])
    for point in points[1::]:
        v1 = builder.create_point(point)
        builder.create_edge_with_vertices(v0,v1)
        v0 = v1
    return curve

## Building random fracture traces

In [ ]:
import random 
import math

# random selection of a point in 2D Bounding Box
def draw_point_in_box(bbox):
    randx = random.uniform(bbox.min().value(0),bbox.max().value(0))
    randy =random.uniform(bbox.min().value(1),bbox.max().value(1))
    return og.Point2D([randx,randy])

# random selection of a point in 2D ring
def draw_point_in_radial_shape(center, rmin,rmax):
    r = rmin + ( rmax - rmin ) * math.sqrt( random.random() )
    theta = ( 2.0 * math.pi ) * random.random()
    randx = center.value( 0 ) + r * math.cos( theta )
    randy =center.value( 1 ) + r * math.sin( theta )
    return og.Point2D([randx,randy])
    
# create one fracture trace
def create_one_fracture_trace(bbox,min_length,max_length):
    tips0= draw_point_in_box(bbox)
    tips1= draw_point_in_radial_shape(tips0,min_length,max_length)
    points= [tips0,tips1]
    return create_edged_curve(points)


box = create_aligned_box2D(100,100)
min_length = 1
max_length = 10
fracture = create_one_fracture_trace(box,min_length,max_length)
print("fracture centrerd on ("+str(fracture.edge_barycenter(0).value(0))+","+str(fracture.edge_barycenter(0).value(1))+")")

## Create a valid Section

In [ ]:
def create_2Dbox_section(box):
    c0 = box.min()
    c1= og.Point2D([box.min().value(0),box.max().value(1)])
    c2 =  box.max()
    c3= og.Point2D([box.max().value(0),box.min().value(1)])
    explicit_modeler = explicit.SectionExplicitModeler(box)
    ec1 = create_edged_curve([c0,c1])
    ec2 = create_edged_curve([c1,c2])
    ec3 = create_edged_curve([c2,c3])
    ec4 = create_edged_curve([c3,c0])
    explicit_modeler.add_curve(ec1)
    explicit_modeler.add_curve(ec2)
    explicit_modeler.add_curve(ec3)
    explicit_modeler.add_curve(ec4)
    return explicit_modeler.build()

def create_fractures(number,box,min_length,max_length):
    _fractures=[]
    for _ in range(number):
        _fractures.append(create_one_fracture_trace(box,min_length,max_length))
    return _fractures 


box = create_aligned_box2D(50,50)
boundaries = create_2Dbox_section(box)

nb_fractures = 100
min_length = 1
max_length = 5
fracture_network = create_fractures(nb_fractures,box,min_length,max_length)

explicit_modeler = explicit.SectionExplicitModeler(box)
explicit_modeler.add_section(boundaries)
for fracture in fracture_network:
    explicit_modeler.add_curve(fracture)
section = explicit_modeler.build()

og.save_section(section, output("fracture_traces.vtm"))

## Remesh the section with triangles

In [ ]:
#remesh
cst_cell_size = min_length
metric = common.ConstantMetric2D(cst_cell_size)
remeshed_sec = simplex.simplex_remesh_section(section,metric)[0]
og.save_section(remeshed_sec, output("remeshed_fracture_traces.vtm"))

## Section extrusion

 * corner --> vertical line
 * line --> vertical surface meshed with square polygons
 * surface --> blocks meshed with prismes


In [ ]:
ext_opt = og.SectionExtruderOptions()
ext_opt.axis_to_extrude = 2
ext_opt.min_coordinate=0
ext_opt.max_coordinate=-5
brep = og.extrude_section_to_brep(remeshed_sec, ext_opt)
og.save_brep(brep,  output("extruded_fracture_traces.vtm"))
print_brep_components(brep)

Previous steps can be merge in the following function with which you can play.

In [ ]:
def remesh_and_extrude_section(section, cst_cell_size, ztop, zbot):
   #remesh
   metric = common.ConstantMetric2D(cst_cell_size)
   remeshed_sec = simplex.simplex_remesh_section(section,metric)[0]
   #extrude
   ext_opt = og.SectionExtruderOptions()
   ext_opt.axis_to_extrude = 2
   ext_opt.min_coordinate=ztop
   ext_opt.max_coordinate=zbot
   brep = og.extrude_section_to_brep(remeshed_sec, ext_opt)
   return brep

def create_fractured_layer(box, nb_fractures, min_length, max_length, ztop, zbot):
    explicit_modeler = explicit.SectionExplicitModeler(box)
    boundaries = create_2Dbox_section(box)
    explicit_modeler.add_section(boundaries)
    fracture_network = create_fractures(nb_fractures,box,min_length,max_length)
    for fracture in fracture_network:
        explicit_modeler.add_curve(fracture)
    section = explicit_modeler.build()
    return remesh_and_extrude_section(section,min_length,ztop,zbot)

nb_fractures = 50
min_length = 1
max_length = 5

box_extent = 10
box2D = create_aligned_box2D(box_extent,box_extent)
ztop = 0
zbot = -0.25 * box_extent
fractured_brep_layer = create_fractured_layer(box2D, nb_fractures, min_length, max_length, ztop, zbot)

og.save_brep(fractured_brep_layer,  output("fractured_brep_layer.og_brep"))
og.save_brep(fractured_brep_layer,  output("fractured_brep_layer.vtm"))